In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt

In [2]:
class Leaf:
    def __init__(self,value):
        self.value = value

In [3]:
class Node:
    def __init__(self,branches,threshold):
        self.branches = branches
        self.threshold = threshold
        
    def get(self,x):
        return self.branches[0] if x < self.threshold else self.branches[1]
        

In [4]:
class Tree:
    def __init__(self,root):
        self.root = root
        
    def predict(self,x):
        item = self.root
        while isinstance(item,Node):
            item = item.get(x)
        return item

In [5]:
r=Node([Leaf('young'),Leaf('old')],18)
t=Tree(r)
print(t.predict(2).value)

young


In [29]:
df=pd.read_csv("iris.csv")

In [7]:
print(t.predict(20).value)

old


In [30]:
class CART:
    def __init__(self,df,y_name,X_names):
        self.df = df
        self.y_name = y_name
        self.X_names = X_names
        self.tree = None
        
    def create_tree(self):
        root = self._node_or_leaf(self.df)
        self.tree = Tree(root)
        return self.tree
    
    def _gini_impurity(self, df):
        unique, counts = np.unique(df[self.y_name].values, return_counts=True)
        N = df[self.y_name].values.ravel().size
        p = counts/N
        print(unique)
        print(p)
        return 1. - np.sum(p)
    
    def _opt_fun(self,df,split_name):
        df1 = df[df[split_name]<split_threshold]
        
    def _node_or_leaf(self,df):
        loss_parent = 1.
        loss_best, split_df, split_threshold = self._loss_best(df)
        if loss_best < loss_parent:
            branches = []
            for i in range(2):
                branches.append(self._node_or_leaf(split_df[i]))
            item = Node(branches,split_threshold)
        else:
            item = Leaf("bla")
        return item
    
    def _loss_best(self,df):
        for name in self.X_names:
            split_threshold = np.median(df[name].values)
            split_df = [df[df[name]<split_threshold],
                        df[df[name]>=split_threshold]]
            loss = self._loss(split_df[0]) + self._loss(split_df[1])
            
        return loss, split_df, split_threshold
    
    def _loss(self,df):
        return self._gini_impurity(df)
            
        
        

In [9]:
df.columns
X_names=["sepal_length","sepal_width"]
df[X_names]

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6
...,...,...
145,6.7,3.0
146,6.3,2.5
147,6.5,3.0
148,6.2,3.4


In [31]:
c = CART(df,"species",X_names)
c.create_tree()

['setosa' 'versicolor' 'virginica']
[0.67123288 0.28767123 0.04109589]
['setosa' 'versicolor' 'virginica']
[0.01298701 0.37662338 0.61038961]
['setosa' 'versicolor' 'virginica']
[0.03508772 0.59649123 0.36842105]
['setosa' 'versicolor' 'virginica']
[0.51612903 0.17204301 0.31182796]
['setosa' 'versicolor' 'virginica']
[0.07142857 0.71428571 0.21428571]
['versicolor' 'virginica']
[0.48275862 0.51724138]
['setosa' 'versicolor' 'virginica']
[0.04166667 0.66666667 0.29166667]
['setosa' 'versicolor' 'virginica']
[0.03030303 0.54545455 0.42424242]
['setosa' 'versicolor' 'virginica']
[0.08333333 0.83333333 0.08333333]
['versicolor' 'virginica']
[0.5 0.5]
['setosa' 'versicolor' 'virginica']
[0.09090909 0.81818182 0.09090909]
['versicolor' 'virginica']
[0.53846154 0.46153846]
['setosa' 'versicolor']
[0.25 0.75]
['versicolor' 'virginica']
[0.85714286 0.14285714]
['versicolor' 'virginica']
[0.75 0.25]
['setosa' 'versicolor']
[0.14285714 0.85714286]
['versicolor']
[1.]
['versicolor' 'virginica']
[